# TensorFlow Playground
## January 2023
### by Michelle (Chelle) Davies
This notebook is my environment to practive using TensorFlow's features. Eventually, I will build more specific projects. For now, there's no (intentional) cohesive narrative with these datasets.

In [1]:
# imports
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.9.1


## Starting with a tutorial on the basics
Source: https://www.tensorflow.org/tutorials/quickstart/beginner

In [2]:
# load the preloaded dataset
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
mnist

<module 'keras.api._v2.keras.datasets.mnist' from '/Users/michelledavies/opt/anaconda3/lib/python3.9/site-packages/keras/api/_v2/keras/datasets/mnist/__init__.py'>

Next, I'm going to build a model. These are the options:
1. Keras Sequential Model
2. Keras Functional API

I'm building a `tf.keras.Sequential` model.

In [4]:
# Build a tf.keras.Sequential model:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

2023-01-29 13:11:25.528408: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# get predictions
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.22331487, -0.37935278, -0.13005681, -0.35523453,  0.5696828 ,
        -0.34033078, -0.626423  ,  0.10205111,  0.34094438, -0.046859  ]],
      dtype=float32)

In [6]:
# The tf.nn.softmax function converts these logits to probabilities for each class:
probabilities = tf.nn.softmax(predictions).numpy()
probabilities

array([[0.08380003, 0.07169317, 0.09199107, 0.0734433 , 0.18519902,
        0.07454608, 0.05599853, 0.11602426, 0.14733261, 0.09997192]],
      dtype=float32)

In [7]:
# Define a loss function for training using losses.SparseCategoricalCrossentropy:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [8]:
loss_fn(y_train[:1], predictions).numpy()

2.5963378

Before training, configure and compile the model using Keras `Model.compile`. Set the optimizer class to adam, set the loss to the `loss_fn` function defined earlier, and specify a metric to be evaluated for the model by setting the metrics parameter to accuracy.

In [9]:
model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])

### Train and evaluate the model
Use the Model.fit method to adjust the model parameters and minimize the loss:

In [10]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2949 - accuracy: 0.9142
Epoch 2/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1438 - accuracy: 0.9570
Epoch 3/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.1085 - accuracy: 0.9678
Epoch 4/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0881 - accuracy: 0.9729
Epoch 5/5
1875/1875 [==============================] - 2s 1ms/step - loss: 0.0736 - accuracy: 0.9775


The `Model.evaluate` method checks the model's performance, usually on a validation set or test set.

In [11]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 0s - loss: 0.0715 - accuracy: 0.9778 - 411ms/epoch - 1ms/step


[0.07153074443340302, 0.9778000116348267]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the TensorFlow tutorials.

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [12]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [13]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.69397127e-08, 1.86762539e-09, 1.69728673e-06, 7.79628826e-05,
        3.32759834e-13, 1.18143415e-08, 9.67167101e-16, 9.99918342e-01,
        9.92806918e-07, 1.04270953e-06],
       [1.31082217e-07, 4.91878192e-04, 9.99467552e-01, 2.01139137e-05,
        1.26456397e-15, 1.94293880e-05, 1.07980256e-07, 3.37800932e-15,
        8.67353265e-07, 8.61450719e-15],
       [2.85797483e-07, 9.97100532e-01, 7.07935658e-04, 4.44425859e-05,
        1.88275626e-05, 1.03889643e-05, 1.44597052e-05, 1.44257257e-03,
        6.60122896e-04, 4.91488549e-07],
       [9.99774635e-01, 3.63051894e-10, 9.35606586e-05, 2.75656156e-08,
        1.07311550e-07, 9.56821077e-06, 2.26367356e-05, 4.74878616e-05,
        1.26392907e-09, 5.20047688e-05],
       [1.53754087e-07, 4.69316461e-11, 3.68167935e-06, 2.38155118e-09,
        9.99710739e-01, 1.62779216e-07, 2.01948046e-06, 1.38170399e-05,
        4.83870508e-08, 2.69416778e-04]], dtype=float32)>

## My Own Experiment
Now, I'm going to make a project with my own data and exploration. I am going to explore creating OCR models with Tensorflow and Keras.